# Advent of Code 2021

I really liked [Peter Norvig's approach](https://github.com/norvig/pytudes/blob/main/ipynb/Advent-2020.ipynb) to writing and tracking solutions, so I'm going to use his method this year.

## Day 0: Imports and Utility Functions

Preparations prior to Day 1:

- Some imports.
- A way to read the day's data file and to print/check the output.
- Some utilities that are likely to be useful.


In [3]:
from collections import namedtuple
from math import prod
import operator
from typing import Callable

In [4]:
def data(day: int, parser=str, sep='\n') -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    sections = open(f'data/advent2021/input{day}.txt').read().rstrip().split(sep)
    return [parser(section) for section in sections]
     
def do(day, *answers) -> dict[int, int]:
    "E.g., do(3) returns {1: day3_1(in3), 2: day3_2(in3)}. Verifies `answers` if given."
    g = globals()
    got = []
    for part in (1, 2):
        fname = f'day{day}_{part}'
        if fname in g: 
            got.append(g[fname](g[f'in{day}']))
            if len(answers) >= part: 
                assert got[-1] == answers[part - 1], (
                    f'{fname}(in{day}) got {got[-1]}; expected {answers[part - 1]}')
    return got


In [5]:
def quantify(iterable, pred=bool) -> int:
    "Count the number of items in iterable for which pred is true."
    return sum(1 for item in iterable if pred(item))

def sliding_window(iterable, window_size) -> list:
    for i in range(len(iterable) - window_size + 1):
        yield iterable[i:i+window_size]

## Day 1: Sonar Sweep

1. How many measurements are larger than the previous measurement?
2. Consider sums of a three-measurement sliding window. How many sums are larger than the previous sum?

In [6]:
in1 = data(1, parser=int)

In [7]:
def day1_1(depths):
    return quantify(sliding_window(depths, 2), lambda window: window[1] > window[0])

In [8]:
def day1_2(depths):
    sums = list(map(sum, sliding_window(depths, 3)))
    return quantify(sliding_window(sums, 2), lambda window: window[1] > window[0])

In [9]:
do(1, 1121, 1065)

[1121, 1065]

## Day 2: Dive!

1. What do you get if you multiply your final horizontal position by your final depth?

In [10]:
in2 = data(2, lambda line: line.split())

In [11]:
Position = tuple[int, int]

def move(position: Position, delta: Position) -> Position:
    return Position(map(operator.add, position, delta))

def navigate(instructions: list[str], position: Position = (0,0)) -> Position:
    for instruction in instructions:
        match instruction:
            case ["forward", n]:
                position = move(position, (int(n), 0))
            case ["down", n]:
                position = move(position, (0, int(n)))
            case ["up", n]:
                position = move(position, (0, -int(n)))
            case _:
                raise ValueError(f"Unmatched instruction: {instruction}")
    return position

In [12]:
def day2_1(instructions): return prod(navigate(instructions))

In [13]:
AimedPosition = namedtuple("AimedPosition", ["horizontal", "depth", "aim"])

def navigate_by_aim(instructions: list[str], position: AimedPosition = AimedPosition(0,0,0)) -> AimedPosition:
    for instruction in instructions:
        match instruction:
            case ["forward", n]:
                position = position._replace(
                    horizontal=position.horizontal + int(n),
                    depth=position.depth + int(n)*position.aim,
                )
            case ["down", n]:
                position = position._replace(aim=position.aim + int(n))
            case ["up", n]:
                position = position._replace(aim=position.aim - int(n))
            case _:
                raise ValueError(f"Unmatched instruction: {instruction}")
    return position

In [14]:
def day2_2(instructions):
    final_position = navigate_by_aim(instructions)
    return final_position.horizontal * final_position.depth

In [15]:
do(2, 1484118, 1463827010)

[1484118, 1463827010]

## Day 3: Binary Diagnostic

Power consumption is gamma rate times epsilon rate, where the gamma rate is the most common bit setting by position in the input and the epsilon rate is the least common bit setting (the bitwise negation).

Life support rating is oxygen generator rating times CO2 scrubber rating. These ratings are found by progressively filtering the list of values to only those with the most/least common bit in the nth position based on the previous bit filtering.

1. What is the power consumption of the submarine?
2. What is the life support rating of the submarine?

In [16]:
Bitlist = list[int]

def most_common_bits(readings: list[str]) -> Bitlist:
    rate = [0] * len(readings[0])
    for reading in readings:
        for i, bit in enumerate(reading):
            rate[i] += 1 if bit == "1" else -1
    return list(map(lambda bit: int(bit >= 0), rate))

def least_common_bits(readings: list[str]) -> Bitlist:
    rate = [0] * len(readings[0])
    for reading in readings:
        for i, bit in enumerate(reading):
            rate[i] += 1 if bit == "1" else -1
    return list(map(lambda bit: int(bit < 0), rate))

def bitlist_to_int(bitlist: Bitlist) -> int:
    val = 0
    for i, bit in enumerate(reversed(bitlist)):
        val += bit * 2**i
    return val

def toggle(bitlist: Bitlist) -> Bitlist:
    return list(map(lambda x: 0 if x else 1, bitlist))

In [17]:
in3 = data(3)

In [18]:
def day3_1(readings):
    gamma = most_common_bits(readings)
    epsilon = least_common_bits(readings)
    return bitlist_to_int(gamma) * bitlist_to_int(epsilon)

In [19]:
def filter_bits(readings: list[str], filter_fn: Callable[[list[str]], int]) -> int:
    filtered = readings
    for i in range(len(readings[0])):
        filter_bits = filter_fn(filtered)
        filtered = list(filter(lambda reading: int(reading[i]) == filter_bits[i], filtered))
        if len(filtered) == 1:
            break
    assert len(filtered) == 1
    return int(filtered[0], base=2)

def o2_rating(readings: list[str]) -> int: return filter_bits(readings, most_common_bits)
def co2_rating(readings: list[str]) -> int: return filter_bits(readings, least_common_bits)



In [20]:
sample3 = [
    '00100',
    '11110',
    '10110',
    '10111',
    '10101',
    '01111',
    '00111',
    '11100',
    '10000',
    '11001',
    '00010',
    '01010',
]
assert o2_rating(sample3) == 23
assert co2_rating(sample3) == 10

In [21]:
def day3_2(readings): return o2_rating(readings) * co2_rating(readings)

In [22]:
do(3, 3429254, 5410338)

[3429254, 5410338]